## News Sentiment

In [9]:
pip install transformer-utils

  Using cached transformers-4.33.1-py3-none-any.whl (7.6 MB)
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------- ----------------------------- 0.4/1.7 MB 8.7 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.7 MB 17.4 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 13.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/293.3 kB ? eta -:--:--
     ------------------------------------- 293.3/293.3 kB 18.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
     ------------ --------------------------- 2.3/7.5 MB 49.0 MB/s eta 0:00:01
     --------------------------- ------------ 5.1/7.5 MB 65.0 MB/s eta 0:00:01
     ---------------------------------------  7.5/7.5 MB 59.9 MB/s eta 0:00:01
     ---------------------------------------  7.5/7.5 MB 59.9 MB/s eta 0:00:01
     ---------------------------------------  7.5/7.5 MB 59.9 MB/s eta 0:00:01
     --------

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\ddg12\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\ddg12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Manage API Key & Use NewsApiClient
I've saved my API key in a config file, I've loaded the file and saved my key to a variable.

In [1]:
import config

APIKEY=config.APIKEY

from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=APIKEY)

# What are the available sources?
I've created a loop to gather all the available sources and save into a dataframe. For clarity sake I've printed the available sources as well as the categories.

In [2]:
import pandas as pd
sources = newsapi.get_sources()
print('How many total sources are available?:',len(sources['sources']))
total_sources = len(sources['sources'])
sources['sources'][0]

id=pd.Series(name='id')
name=pd.Series(name='name')
description=pd.Series(name='description')
url=pd.Series(name='url')
category=pd.Series(name='category')
language=pd.Series(name='language')
country=pd.Series(name='country')
#column_names = ['id', 'name', 'description', 'url', 'category', 'language', 'country']

for idx,outlet in enumerate(sources['sources']):
    id[idx]=outlet['id']
    name[idx] = outlet['name']
    description[idx] = outlet['description']
    url[idx] = outlet['url']
    category[idx] = outlet['category']
    language[idx] = outlet['language']
    country[idx] = outlet['country']
sourcedf=pd.concat([id,name,description,url,category,language,country],axis=1)

print('What are the categories? ', sourcedf['category'].unique())

How many total sources are available?: 128
What are the categories?  ['general' 'business' 'technology' 'sports' 'entertainment' 'health'
 'science']


# Filter the available sources
Requirements: English Language, US Country

In [3]:
sourcedf= sourcedf[(sourcedf['language']=='en') & (sourcedf['country']=='us')]
sourcedf.head()

,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
3,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",http://www.aljazeera.com,general,en,us
6,ars-technica,Ars Technica,The PC enthusiast's resource. Power users and ...,http://arstechnica.com,technology,en,us
8,associated-press,Associated Press,The AP delivers in-depth coverage on the inter...,https://apnews.com/,general,en,us
10,axios,Axios,Axios are a new media company delivering vital...,https://www.axios.com,general,en,us


# What are the business sources?
First we isolate the business sources based on the category variable.
I've displayed the number of business sources as a sanity check, as well as tested that the indexing was working correctly.

In [4]:
business_sources = sourcedf[sourcedf['category']=='business']
business_sources = business_sources.reset_index()
display(business_sources)
print('Number of sources: ',len(business_sources))
business_sources['id'][0]

,index,id,name,description,url,category,language,country
0,16,bloomberg,Bloomberg,"Bloomberg delivers business and markets news, ...",http://www.bloomberg.com,business,en,us
1,18,business-insider,Business Insider,Business Insider is a fast-growing business si...,http://www.businessinsider.com,business,en,us
2,36,fortune,Fortune,Fortune 500 Daily and Breaking Business News,http://fortune.com,business,en,us
3,117,the-wall-street-journal,The Wall Street Journal,WSJ online coverage of breaking news and curre...,http://www.wsj.com,business,en,us


Number of sources:  4


'bloomberg'

# Get 30 Days Ago Date in the Proper Format
For the loop below to work (on the free tier) we need to only check back 30 days. We will use the datetime package to get todays date and 30 days prior.

In [5]:
# Get 30 Days Ago
import datetime
from datetime import date, timedelta
print('Todays Date: ', date.today().isoformat())
print('30 Days Ago: ', (date.today()-timedelta(days=30)).isoformat())
startdate=(date.today()-timedelta(days=30)).isoformat()

Todays Date:  2023-09-10
30 Days Ago:  2023-08-11


# Loop through all articles 
There are two layers to this loop, the outer layer loops through the number of business sources (4), while the inner loop gets all the articles from that business source and collects the data.


In [6]:
# Init Variables
Source=pd.Series(name='source')
author=pd.Series(name='author')
title=pd.Series(name='title')
description=pd.Series(name='description')
url=pd.Series(name='url')
publishedAt=pd.Series(name='publishedAt')
content=pd.Series(name='content')
# Use Counter
counter=0
# Loop through each source 
for source in business_sources['id']:
    news_articles = newsapi.get_everything(q='apple',
                                          sources=source,
                                          from_param=startdate,
                                          )
    # Gather data from each article
    for idx, article in enumerate(news_articles['articles']):
        Source[counter] = article['source']['id']
        author[counter] = article['author']
        title[counter] = article['title']
        description[counter] = article['description']
        url[counter] = article['url']
        publishedAt[counter] = article['publishedAt']
        content[counter] = article['content']
        #author=pd.concat([Source,article['author']],axis=1)
        #for details in article:
        #    print(details['title'])
        counter = counter + 1
# Combine 
all_articles = pd.concat([Source, author, title, description, url, publishedAt, content],axis=1)
all_articles.head()

,source,author,title,description,url,publishedAt,content
0,business-insider,Lakshmi Varanasi,Apple is reportedly working on the Watch's big...,Apple is planning to release a new model of th...,https://www.businessinsider.com/apple-working-...,2023-08-14T16:03:56Z,Apple has a major redesign of the Apple Watch ...
1,business-insider,htan@insider.com (Huileng Tan),The billionaire founder of a key Apple supplie...,He said no foreign investor would dare invest ...,https://www.businessinsider.com/foxconn-terry-...,2023-08-28T10:21:20Z,"Terry Gou, the billionaire founder of Foxconn ..."
2,business-insider,Theron Mohamed,4 Big Tech giants have plowed over $1 trillion...,Apple poured over $600 billion into buybacks i...,https://markets.businessinsider.com/news/stock...,2023-08-30T10:39:42Z,Apple CEO Tim Cook.Richard Drew/AP Images\r\n<...
3,business-insider,Pete Syme,"Apple, Amazon, and Meta among 6 tech giants de...","The European Union named Alphabet, Amazon, App...",https://www.businessinsider.com/tiktok-and-met...,2023-09-06T11:55:50Z,"Google, Apple, Facebook, Amazon, and Microsoft..."
4,business-insider,Business Insider,Big Tech salaries revealed: This is what devel...,Big tech salaries unveil earnings of engineers...,https://www.businessinsider.com/big-tech-salar...,2023-08-23T16:47:20Z,Business Insider analyzed salary data for work...


# Sentiment Analysis
The time has finally come for the namesake of this workbook... Sentiment Analysis.

We will attempt a few different methods for sentiment analysis. We'll be using a BERT Transformer model (The "T" in BERT), which is a natural language processing model released by google that uses deep learning (convolutional neural networks and recurrent neural networks) to "understand" natural language.

In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
# Load BERT Model
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# preprocess the text
text = "This is a great movie! I really enjoyed it."
input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])

# classify the text
with torch.no_grad():
  logits = model(input_ids)

# get the predicted label
label_idx = logits.argmax().item()

# map the label index to a label
label_map = {0: "negative", 1: "positive"}
sentiment = label_map[label_idx]
print(sentiment)

ModuleNotFoundError: No module named 'transformers.utils'

In [ ]:
pd.Timestamp.today()

Timestamp('2023-09-09 23:59:35.684985')

# Download Stock Data

In [ ]:
import yfinance as yf
